In [1]:
# Import required libraries into the jupyter notebook

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

Beginning Code blocks to do the things I want it to do

In [19]:
# Step to read in weather data and pizza data csv files from the data directory
weather_file = pd.read_csv("data/alexandria_weather_data.csv")
pizza_file = pd.read_csv("data/alexandria_pizza_data.csv")


In [14]:
# Display head of weather file to show successful read of file
weather_file.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,41001,3/1/2021,55.0,32.4,41.6,55.0,23.9,35.3,28.8,62.0,...,12.9,6,NaN,2021-03-01T07:09:32,2021-03-01T18:30:39,0.58,"Rain, Partially cloudy",Clearing in the afternoon with early morning r...,rain,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."
1,41001,3/2/2021,44.4,23.8,33.1,44.4,16.6,28.6,16.5,52.2,...,17.1,7,NaN,2021-03-02T07:08:04,2021-03-02T18:31:42,0.62,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
2,41001,3/3/2021,62.1,26.8,43.8,62.1,22.7,42.2,26.6,54.2,...,16.6,7,NaN,2021-03-03T07:06:36,2021-03-03T18:32:45,0.65,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
3,41001,3/4/2021,51.9,30.0,41.2,51.9,23.2,36.7,25.9,57.3,...,16.9,7,NaN,2021-03-04T07:05:07,2021-03-04T18:33:47,0.69,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
4,41001,3/5/2021,47.1,25.0,35.1,42.2,18.4,31.0,18.9,54.5,...,17.2,7,NaN,2021-03-05T07:03:38,2021-03-05T18:34:50,0.75,Clear,Clear conditions throughout the day.,clear-day,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."


In [20]:
# Diesply head of pizza date file to show successful read of file
pizza_file.head()

,BusDate,OrderID,OtotSubTotal,OrdServiceType
0,3/1/2021,2,20.74,3
1,3/2/2021,3,19.60,1
2,3/3/2021,5,20.97,1
3,3/4/2021,6,9.99,1
4,3/5/2021,7,22.98,1


In [23]:
# Joining both files into one data element to begin analysis

left = pizza_file.set_index('BusDate')
right = weather_file.set_index('datetime')

left.join(right, lsuffix='_p', rsuffix="_w")


# - FOLLOW-UP - this is not working right, something looks to be wrong with the pizza data source file.

,OrderID,OtotSubTotal,OrdServiceType,name,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
BusDate,,,,,,,,,,,,,,,,,,,,,
3/1/2021,2,20.74,3,41001.0,55.0,32.4,41.6,55.0,23.9,35.3,...,12.9,6.0,NaN,2021-03-01T07:09:32,2021-03-01T18:30:39,0.58,"Rain, Partially cloudy",Clearing in the afternoon with early morning r...,rain,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."
3/2/2021,3,19.60,1,41001.0,44.4,23.8,33.1,44.4,16.6,28.6,...,17.1,7.0,NaN,2021-03-02T07:08:04,2021-03-02T18:31:42,0.62,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
3/3/2021,5,20.97,1,41001.0,62.1,26.8,43.8,62.1,22.7,42.2,...,16.6,7.0,NaN,2021-03-03T07:06:36,2021-03-03T18:32:45,0.65,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
3/4/2021,6,9.99,1,41001.0,51.9,30.0,41.2,51.9,23.2,36.7,...,16.9,7.0,NaN,2021-03-04T07:05:07,2021-03-04T18:33:47,0.69,Clear,Clear conditions throughout the day.,clear-day,"KI69,F8613,KLUK,E5157,72421093814,KCVG,7242979..."
3/5/2021,7,22.98,1,41001.0,47.1,25.0,35.1,42.2,18.4,31.0,...,17.2,7.0,NaN,2021-03-05T07:03:38,2021-03-05T18:34:50,0.75,Clear,Clear conditions throughout the day.,clear-day,"KI69,KLUK,E5157,72421093814,KCVG,72429793812,A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2/8/2277,170,29.99,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2/9/2277,171,40.70,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2/10/2277,172,55.21,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
